In [1]:
import torchvision
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torch
from time import time
from torch import nn, optim

In [2]:
trainset = torchvision.datasets.FashionMNIST(root='./data', download = True, train = True, transform = transforms.Compose([transforms.ToTensor()]))

In [3]:
data_loader = torch.utils.data.DataLoader(trainset, batch_size= 10,shuffle = True)

In [4]:
image, label = next(iter(trainset))

In [5]:
import torch.nn.functional as F
import torch.nn as nn
import random

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand = torch.randint(0, 10, (3,)).cuda()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 12, 5)
        
        self.fc1 = nn.Linear(193, 120)
        self.fc2 = nn.Linear(120, 60)
        self.out = nn.Linear(60, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        rand = torch.randint(0, 10, (x.shape[0],1)).cuda()
        x = torch.cat((x,rand),1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        x = F.softmax(x, dim=1)
        out = x.argmax(dim=1)
        out = out + rand
        return (x,out)


net = Net()

In [6]:
import torch.optim as optim

# defining the optimizer
optimizer = optim.Adam(net.parameters(), lr=0.01)
# defining the loss function
criterion = nn.CrossEntropyLoss()

net = net.cuda()
criterion = criterion.cuda()
    
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=193, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)


In [7]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [8]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(data_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.344
[1,  4000] loss: 2.360
[1,  6000] loss: 2.363
[2,  2000] loss: 2.361
[2,  4000] loss: 2.364
[2,  6000] loss: 2.359
[3,  2000] loss: 2.362
[3,  4000] loss: 2.362
[3,  6000] loss: 2.359
[4,  2000] loss: 2.360
[4,  4000] loss: 2.360
[4,  6000] loss: 2.363
[5,  2000] loss: 2.362
[5,  4000] loss: 2.360
[5,  6000] loss: 2.361
[6,  2000] loss: 2.362
[6,  4000] loss: 2.363
[6,  6000] loss: 2.359
[7,  2000] loss: 2.361
[7,  4000] loss: 2.361
[7,  6000] loss: 2.361
[8,  2000] loss: 2.358
[8,  4000] loss: 2.362
[8,  6000] loss: 2.363
[9,  2000] loss: 2.361
[9,  4000] loss: 2.358
[9,  6000] loss: 2.364
[10,  2000] loss: 2.363
[10,  4000] loss: 2.361
[10,  6000] loss: 2.360
Finished Training


In [11]:
image, label = next(iter(trainset))

In [14]:
output = net(image.unsqueeze(0).cuda())


In [15]:
print(output)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
